# Learning CartPole with Q-Learning!
This notebook will tackle the game of CartPole using a value-based reinforcement learning method known as Q-Learning! The CartPole environment is readily available through OpenAI's gym module.

## Q-Learning
The idea of value-based reinforcement learning methods are to indirectly find the optimal-policy of a task by first estimating the worth of a particular state under some policy governed by these values. More succinctly, the expected future returns of each state are calculated and a policy is found by (usually) acting greedily with respect to these values.

Assuming that our system is a Markov Decision Process (MDP), the expected reward for being in a particular state s and following some fixed policy $\pi$ can be expressed by the Bellman Equation:

$$V^{\pi}(s) = \sum_{a} \pi (s, a) \sum_{s'} \mathcal{P}_{s s'}^{a} \Big[ \mathcal{R}_{s s'}^{a} + \gamma V^{\pi}(s') \Big] $$

where $\mathcal{R}_{s s'}^{a}$ is the reward function, $\gamma$ is the discount factor, $\mathcal{P}_{s s'}^{a}$ are the transition probabilties, and $V^{\pi}(*)$ is the state-value function under a policy, $\pi$.

By choosing the action that gives rise to the highest expected return (acting greedily), we can then write the Bellman Equation that mimics this behaviour, known as the Bellman Optimality Equation.

$$V^{\pi}(s) = \max_a\sum_{s'} \mathcal{P}_{s s'}^{a} \Big[ \mathcal{R}_{s s'}^{a} + \gamma V^{\pi}(s') \Big] $$

Generally, if the MDP is fully-defined it is possible to solve for these state-values for all states using dynamic programming. Unfortunately, in most cases, the MDP is not completely known and therefore other methods are needed to solve the Bellman Equations. 

One such method of solving the Bellman Optimality Equation is Q-learning. It is an off-policy TD-learning method that uses gradient descent updates at every step when the agent is exploring the environment. 

$$Q_{t+1}^\pi(s_n, a_n) = Q_t^\pi(s_n, a_n) + \alpha \bigg[\mathcal{R}_{s s'}^{a} + \gamma \max_{a'} Q_t^\pi(s_{n+1}, a') - Q_t^\pi(s_n, a_n)\bigg]$$

When dealing with a discrete state-space, action-values can be stored in a table -- but when dealing with continous state-spaces, this becomes intractable. As a result, the action-value function has to be approximated. Neural networks are great function-approximators, and such they are a good tool to use for this task.

In [8]:
import numpy as np
import random
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from q_learning import PolicyNetwork, QAgent

In [13]:
env = gym.make("CartPole-v0")
    
state_space = env.observation_space
action_space = env.action_space

batch_size = 256
hidden_size = 32
memory_flush = 2000

discount = 0.99
episodes = 1000

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [14]:
number_of_runs = 10
run_rewards = []

for run in range(number_of_runs):
    policy_network = PolicyNetwork(state_space, action_space, hidden_size).double()
    target_network = PolicyNetwork(state_space, action_space, hidden_size).double()

    agent = QAgent(policy_network, target_network, epsilon=0.01)

    criterion = nn.MSELoss()
    optimiser = torch.optim.RMSprop(policy_network.parameters())
    
    episode_rewards = []
    for episode in range(episodes):
        state = torch.tensor(env.reset())
        score = 0
        loss = 0

        if episodes % 10 == 0:
            target_network.load_state_dict(policy_network.state_dict())

        done = False
        while not done:
            optimiser.zero_grad()

            action = agent.select_action(state, action_space)
            next_state, reward, done, info = env.step(action.data.numpy()[0, 0])
            next_state = torch.tensor(next_state).type(torch.DoubleTensor)

            reward = torch.tensor(reward).type(torch.FloatTensor).view(-1, 1)

            score += reward

            agent.memory.append((state, action, reward, next_state, done))

            if len(agent.memory) > memory_flush:
                del agent.memory[0]

            if len(agent.memory) > batch_size:
                minibatch = random.sample(agent.memory, batch_size)
                batch_state, batch_action, batch_reward, batch_next_state, batch_done = zip(*minibatch)

                batch_state = torch.stack(batch_state)
                batch_action = torch.stack(batch_action).view(-1, 1)
                batch_reward = torch.stack(batch_reward).view(-1, 1)
                batch_next_state = torch.stack(batch_next_state)

                Q_next_max, _ = torch.max(target_network.forward(batch_next_state).detach(), dim=1)
                Q_next_max = Q_next_max.view(-1, 1).detach()

                discounted_Q = (discount * Q_next_max).type(torch.FloatTensor)
                batch_reward = batch_reward.type(torch.FloatTensor)

                Q_current = policy_network.forward(batch_state)
                Q_target = Q_current.clone()

                for i in range(len(Q_target)):
                    if not batch_done[i]:
                        Q_target[i, batch_action[i][0]] = batch_reward[i] + discounted_Q[i]

                    else:
                        Q_target[i, batch_action[i][0]] = batch_reward[i]

                loss = criterion(Q_current, Q_target)

                loss.backward()
                optimiser.step()

            state = next_state

            if done and run == 0 and episode % 20 == 0:
                print("Episode %d -> Loss: %.4f\t Reward: %d \t(eps: %.4f)" % (episode, loss, score, agent.epsilon))
                break

        episode_rewards.append(score.data.numpy()[0, 0])
    
    run_rewards.append(episode_rewards)

Episode 0 -> Loss: 0.0000	 Reward: 9 	(eps: 0.0100)
Episode 20 -> Loss: 0.0000	 Reward: 10 	(eps: 0.0100)
Episode 40 -> Loss: 1008.2428	 Reward: 31 	(eps: 0.0100)
Episode 60 -> Loss: 613.7448	 Reward: 37 	(eps: 0.0100)


KeyboardInterrupt: 

In [ ]:
%matplotlib inline
def moving_average(a, n=20) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

means = np.mean([moving_average(rewards) for rewards in run_rewards], axis=0)
stds = np.std([moving_average(rewards) for rewards in run_rewards], axis=0)

plt.plot(range(len(means)), means, color='red')

plt.fill_between(range(len(means)), means, means + stds, color='blue', alpha=.25)
plt.fill_between(range(len(means)), means, means - stds, color='blue', alpha=.25)

plt.show()